In [ ]:
from core.trainer import train_gfn

spin_js = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
single_vertex_params = [{
    "spin_j" : j,
    "sf_model" : "single_vertex_model",
    "main_layer_hidden_nodes" : (
        int(2*j+1)*5, max( int(2*j+1)*5 // 2, 10), 6
    ),
    "branch1_hidden_nodes" : (),
    "branch2_hidden_nodes" : (),
    "activation" : "swish",
    "exploration_rate" : 0.5, #try 0
    "training_fraction_from_back_traj" : 0.0,
    "learning_rate" : 0.001,
    "batch_size" : int(1e3),
    "n_iterations" : int(1e4),
    "evaluation_batch_size" : int(1e6),
    "generate_samples_every_m_training_samples" : int(1e6),
} for j in spin_js]

# star_params = [{
#     "spin_j" : j,
#     "sf_model" : "star_model",
#     "main_layer_hidden_nodes" : (
#         int(2*j+1)*20, max( int(2*j+1)*20 // 2, 40), max( int(2*j+1)*20 // 4, 30), 21
#     ),
#     "branch1_hidden_nodes" : (),
#     "branch2_hidden_nodes" : (),
#     "activation" : "swish",
#     "exploration_rate" : 0.5,
#     "training_fraction_from_back_traj" : 0.5,
#     "learning_rate" : 0.001,
#     "batch_size" : int(1e3),
#     "n_iterations" : int(1e4),
#     "evaluation_batch_size" : int(5e4),
#     "generate_samples_every_m_training_samples" : int(1e6),
# } for j in spin_js]


In [ ]:
losses = []
for params in single_vertex_params:
# for params in star_params:
    losses.append(train_gfn(**params, model_save_dir="."))

In [6]:
import pandas as pd
df = pd.read_feather("./generated_samples/epoch_1000_after_learn_from_1000000_train_samples.feather")
df.head()

,intertwiner 5,intertwiner 4,intertwiner 3,intertwiner 2,intertwiner 1
0,1,0,0,1,0
1,0,1,0,0,0
2,0,1,0,0,1
3,1,1,1,1,1
4,1,1,1,1,1


In [ ]:
from core.trainer import train_gfn

params = {
    "spin_j" : 3.5,
    "sf_model" : "single_vertex_model",
    "main_layer_hidden_nodes" : (
#         128, 128, 64, 64, 32, 32, 16, 16, 8, 8
        128, 32, 32, 8
    ),
    "branch1_hidden_nodes" : (),
    "branch2_hidden_nodes" : (),
    "activation" : "swish",
    "exploration_rate" : 0.5,
    "training_fraction_from_back_traj" : 0.5,
    "learning_rate" : 0.001,
    "batch_size" : int(1e3),
    "n_iterations" : int(1e4),
    "evaluation_batch_size" : int(5e4),
    "generate_samples_every_m_training_samples" : int(1e6),
}

ave_losses = train_gfn(**params)

In [ ]:
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 1.9609801666379112
Nth iteration: 2000 Trained on K samples: 2000000 Average Loss: 1.065302608731616
Nth iteration: 3000 Trained on K samples: 3000000 Average Loss: 0.59914980090523229
Nth iteration: 4000 Trained on K samples: 4000000 Average Loss: 0.38473474757335896
Nth iteration: 5000 Trained on K samples: 5000000 Average Loss: 0.35055703172378194
Nth iteration: 6000 Trained on K samples: 6000000 Average Loss: 0.32772723318299496
Nth iteration: 7000 Trained on K samples: 7000000 Average Loss: 0.29690569581730686
Nth iteration: 8000 Trained on K samples: 8000000 Average Loss: 0.39323040688549454
Nth iteration: 9000 Trained on K samples: 9000000 Average Loss: 0.28258110530944874
Nth iteration: 10000 Trained on K samples: 10000000 Average Loss: 0.25508918216647825

In [ ]:
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 1.3386958754352229
Nth iteration: 2000 Trained on K samples: 2000000 Average Loss: 1.0995962435696534
Nth iteration: 3000 Trained on K samples: 3000000 Average Loss: 0.85274413082630662
Nth iteration: 4000 Trained on K samples: 4000000 Average Loss: 0.45730040409348371
Nth iteration: 5000 Trained on K samples: 5000000 Average Loss: 0.60429376325367323
Nth iteration: 6000 Trained on K samples: 6000000 Average Loss: 0.382636286629316
Nth iteration: 7000 Trained on K samples: 7000000 Average Loss: 0.42668693697825882
Nth iteration: 8000 Trained on K samples: 8000000 Average Loss: 0.39781253586940124
Nth iteration: 9000 Trained on K samples: 9000000 Average Loss: 0.29250289648957822
Nth iteration: 10000 Trained on K samples: 10000000 Average Loss: 0.245781074454583

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
for j, l in zip(spin_js, losses):
    ax.plot(l, label=j)
ax.legend()
# ax.set_ylim((0, 2))

In [ ]:
spins = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
# nodes = {
#     0.5: (15, 10),
#     1: (20, 10),
#     1.5: (25, 10),
#     2: (30, 15),
#     2.5: (35, 15),
#     3: (40, 15),
#     3.5: (45, 20),
#     4: (50, 20),
#     4.5: ()
# }

enc_dim = [int(2*j+1)*20 for j in spins]
enc_dim

# train_gfn(
#     spin_j=3.5, 
#     sf_model="single_vertex_model",
#     main_layer_hidden_nodes=(30, 20), 
#     branch1_hidden_nodes=(), 
#     branch2_hidden_nodes=(),
#     activation="swish",
#     exploration_rate=0.5,
#     training_fraction_from_back_traj=0.0,
#     learning_rate=0.0005,
#     batch_size=int(1e3),
#     n_iterations=int(1e4),
#     evaluation_batch_size=int(1e6),
#     generate_samples_every_m_training_samples=int(1e6),
# )

In [ ]:
[int(2*j+1)*20//4 for j in spins]

In [ ]:
import os
from datetime import datetime

from core.runner import Runner
from core.environment import SingleVertexSpinFoam, StarModelSpinFoam

def train_gfn(
    spin_j=3.5, 
    sf_model="single_vertex_model",
    main_layer_hidden_nodes=(30, 20), 
    branch1_hidden_nodes=(), 
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    training_fraction_from_back_traj=0.0,
    learning_rate=0.0005,
    batch_size=int(1e3),
    n_iterations=int(1e4),
    evaluation_batch_size=int(1e6),
    generate_samples_every_m_training_samples=int(1e6),
):
    """
    Train a GFlowNet agent to sample grid coordinates with probabilities proportional 
    to a target reward distribution. The SpinFoam model defines the grid environment 
    which the agent interacts with. The grid coordinates then correspond to an ordered 
    set of boundary intertwiners while the square of the SpinFoam amplitudes define the 
    reward for each grid coordinate. This results in grid coordinates with higher rewards
    being sampled more frequently so that the corresponding boundary intertwiners can be
    used for the calculation of observables downstream.
    
    Parameters
    ----------
    
    spin_j: (float)
                - Half-integer boundary spin. All boundary spins are fixed to this value
                - Determines the grid length. All sides of the grid have length: 
                  grid_length = 2*spin_j + 1
                
    sf_model: (str)
                - Name specifying the particular spinfoam model. Currently, can only be
                  "single_vertex_model" or "star_model". Custom SpinFoam classes can also
                  be implemented. 
                - Determines the number of boundary intertwiners which in turn determines the
                  dimensionality of the grid. 
                - E.g. "single_vertex_model" -> 5-dimensional grid: grid_dim = 5, 
                       "star_model" -> 20-dimensional grid: grid_dim = 20
                  
    main_layer_hidden_nodes: (tuple)
                - Tuple of integers defining the number of hidden nodes per layer after the 
                  input layer in the main neural network branch. This main branch splits into
                  two branches corresponding to backward and forward action probabilities. 
                  So, the last layer of the main branch connects to both the first layers of
                  the backward-action and forward-action branches.
                - Input layer contains grid_length*grid_dim nodes because of the one-hot 
                  encoding of each grid coordinate
                - E.g. (128, 64, 32) -> 3 layers after the input layer in the main branch: 
                                        128 nodes for the first one, 64 for the next, and 32
                                        for the last
                                        
                Ex: spin=3.5, sf_model="single_vertex_model"
                 -> grid_length=8, grid_dim=5
                 -> input layer = 8*5 = 40
                 -> output layer = 5, 6
                 -> main_layer_hidden_nodes = (64, 40, 10), (30, 15)
                 
                 spin=3.5, sf_model="star_model"
                 -> input layer = 8*20 = 160
                 -> output layer = 20, 21
                 -> main_layer_hidden_nodes = (120, 60, 30), (200, 150, 100, 50), (120, 30)
                
                                        
    branch1_hidden_nodes: (tuple)
                - Tuple of integers (or empty tuple) defining the number of hidden nodes per 
                  layer in the backward-action branch before the output layer
                - Output layer for this branch contains grid_dim nodes
                - E.g. (16, 8) -> 2 layers before the output layer:
                                  16 for the first one, then 8 for the next
                       () -> there is only the output layer in this branch
                       
    branch2_hidden_nodes: (tuple)
                - Tuple of integers (or empty tuple) defining the number of hidden nodes per 
                  layer in the forward-action branch before the output layer
                - Output layer for this branch contains grid_dim+1 nodes
                - E.g. (16, 8) -> 2 layers before the output layer:
                                  16 for the first one, then 8 for the next
                       () -> there is only the output layer in this branch
                       
    activation: (str)
                - Name of the nonlinear functions used for each layer other than the output layers
                - E.g.: elu, exponential, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softmax, 
                        softplus, softsign, swish, tanh
                - tanh, swish, relu
                        
    exploration_rate: (float)
                - Number from 0 to 1 representing the amount of random noise added to the forward 
                  actions. More noise encourages the agent to explore the grid environment further
                - Given by: New_action_proabilities = (
                                exploration_rate*noise_action_probabilities + 
                                (1 - exploration_rate)*current_action_probabilities
                            )
                - I.e.: exploration_rate = 0 -> no noise added
                        exploration_rate = 1 -> actions are just from random noise
                        
                        =0.5

                        
    training_fraction_from_back_traj: (float)
                - Number from 0 to 1 representing the fraction of the training set which is 
                  generated from backward trajectories. The rest is generated from forward trajectories.
                - I.e.: n_batch_backwards = training_fraction_from_back_traj*batch_size
                        n_batch_forwards = (1 - training_fraction_from_back_traj)*batch_size
                        
                    "single_vertex" -> 0.0, "star_model" -> 0.5
                        
    learning_rate: (float)
                - Represents the amount of the gradient used by the optimizer to update the neural 
                  network parameters per training iteration 
                  
    batch_size: (int)
                - Number of trajectories generated per training iteration. These trajectories form
                  the training set that the agent learns from. 
                  
                  = 1000
                  
    n_iterations: (int)
                - Number of training iterations for the agent
                
                = 10000
                
    evaluation_batch_size: (int)
                - Number of grid coordinates that the agent generates and stores for later evaluation
                
                = 10000
                
    generate_samples_every_m_training_samples: (int)
                - Number of trajectories generated during training before the agent samples new 
                  grid coordinates and stores them
                  
                = batch_mcmc
    """
    if sf_model == "single_vertex_model":
        spinfoam_model = SingleVertexSpinFoam(spin_j=spin_j)
    elif sf_model == "star_model":
        spinfoam_model = StarModelSpinFoam(spin_j=spin_j)
    else:
        raise ValueError(
            "Spinfoam model not yet implemented. "
            "Custom Spinfoam class can be made."
        )
        
    training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
    directory_for_generated_samples = (
        f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
    )
    
    runner = Runner(
        spinfoam_model=spinfoam_model,
        main_layer_hidden_nodes=main_layer_hidden_nodes,
        branch1_hidden_nodes=branch1_hidden_nodes,
        branch2_hidden_nodes=branch2_hidden_nodes,
        activation=activation,
        exploration_rate=exploration_rate,
        training_fraction_from_back_traj=training_fraction_from_back_traj,
        learning_rate=learning_rate
    )
    
    ave_losses = runner.train_agent(
        batch_size, n_iterations, 
        evaluation_batch_size, generate_samples_every_m_training_samples,
        directory_for_generated_samples
    )
    
    return ave_losses
    
    


# Single Vertex

In [ ]:
ave_losses = train_gfn(
    spin_j=3.5, 
    sf_model="single_vertex_model",
    main_layer_hidden_nodes=(30, 20), 
    branch1_hidden_nodes=(), 
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    training_fraction_from_back_traj=0.0,
    learning_rate=0.0005,
    batch_size=int(1e3),
    n_iterations=int(1e4),
    evaluation_batch_size=int(1e6),
    generate_samples_every_m_training_samples=int(1e6),
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ave_losses)

# Star Model

In [ ]:
ave_losses = train_gfn(
    spin_j=3.5, 
    sf_model="star_model",
    main_layer_hidden_nodes=(128, 64, 64), 
    branch1_hidden_nodes=(), 
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    training_fraction_from_back_traj=0.5,
    learning_rate=0.0005,
    batch_size=int(1e3),
    n_iterations=int(1e4),
    evaluation_batch_size=int(1e6),
    generate_samples_every_m_training_samples=int(1e6),
)

In [ ]:
plt.plot(ave_losses)

In [ ]:
from core.runner import Runner
from core.environment import SingleVertexSpinFoam

spin_j = 3.5
sf_model = "single_vertex_model"

runner = Runner(
    spinfoam_model=SingleVertexSpinFoam(spin_j=spin_j),
    main_layer_hidden_nodes=(30, 20),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    training_fraction_from_back_traj=0.0,
    learning_rate=0.0005
)

In [ ]:
import os
from datetime import datetime


training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)

batch_size = int(1e3)
n_iterations = int(1e3)
# n_iterations = int(300)
evaluation_batch_size = int(10)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

# Star Model

In [ ]:
from core.runner import Runner
from core.environment import StarModelSpinFoam

spin_j = 3.5
sf_model = "star_model"

runner = Runner(
    spinfoam_model=StarModelSpinFoam(spin_j=spin_j),
    main_layer_hidden_nodes=(128, 64, 64),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    training_fraction_from_back_traj=0.1,
    learning_rate=0.0005
)

In [ ]:
import os
from datetime import datetime


training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)

batch_size = int(1e3)
n_iterations = int(1e3)
# n_iterations = int(300)
evaluation_batch_size = int(10)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

----------------------------------

# Previous tests

In [ ]:
from core.runner import Runner
from core.environment import SingleVertexSpinFoam

spin_j = 3.5
sf_model = "single_vertex_model"

runner = Runner(
    spinfoam_model=SingleVertexSpinFoam(),
    spin_j=spin_j,
    main_layer_hidden_nodes=(30, 20),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.0005
)

In [ ]:
import os
from datetime import datetime


training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)


batch_size = int(1e3)
n_iterations = int(1e3)
# n_iterations = int(300)
fraction_of_training_from_back_trajectories = 0.0
evaluation_batch_size = int(2e5)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

In [ ]:
0.999 Max


Nth iteration: 1 Average Loss: 10.313235918730944
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 3.8410917472658523
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 3.2507308831058417
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 4.3149855892960058
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 3.5061072391776023
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 2.4787907542318872

In [ ]:
0.99 Max

Nth iteration: 1 Average Loss: 13.300160685018087
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 5.0689388321024333
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 3.6152086536530477
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 3.0158918708159232
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 2.2089659154210057
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 1.4632014533468054

In [ ]:
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 15.041544509167547
            
Nth iteration: 1 Average Loss: 27.833997273130787
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 14.439725408179275
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 8.581400663157444
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 4.7001221032761746
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 3.9371769914849057
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 2.5361100055773012

In [ ]:
Max

Nth iteration: 200 Trained on K samples: 200000 Average Loss: 3.6238780132468866

Nth iteration: 1 Average Loss: 10.613150147572496
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 3.5966320343431581
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 4.494772864296265
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 4.1429286237455232
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 3.9257716045337037
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 3.7467994673681444
            
Nth iteration: 1 Average Loss: 10.208967923761881
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 4.71511010701389
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 5.5712986666670261
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 4.24535714700103
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 3.7134989553479949
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 3.1275138985696822

In [ ]:
Mean

Nth iteration: 200 Trained on K samples: 200000 Average Loss: 24.56604181036295

In [ ]:
import tensorflow as tf

single_vertex_amplitudes = runner.env.single_vertex_amplitudes

In [ ]:
ave_abs = tf.math.reduce_mean(tf.math.abs(single_vertex_amplitudes))
ave_abs

In [ ]:
2*tf.math.log(ave_abs)

In [ ]:
-22, -12, -14, -19
-49, -52, -61, -84

In [ ]:
sq_scale = tf.math.reduce_sum(tf.math.square(single_vertex_amplitudes))
sq_scale

In [ ]:
prev_log_z = tf.math.log(sq_scale)
prev_log_z

In [ ]:
max_abs = tf.math.reduce_max(tf.math.abs(single_vertex_amplitudes))
max_abs

In [ ]:
2*tf.math.log(max_abs)

In [ ]:
max_sq = tf.math.reduce_max(tf.math.square(single_vertex_amplitudes))
max_sq

In [ ]:
tf.math.log(max_sq)

In [ ]:
min_abs = tf.math.reduce_min(tf.math.abs(single_vertex_amplitudes))
min_abs

In [ ]:
2*tf.math.log(min_abs)

In [ ]:
-52.23322188306788*0.99

In [ ]:
from core.runner import Runner
from core.environment import SingleVertexSpinFoam

spin_j = 6.0
sf_model = "single_vertex_model"

runner = Runner(
    spinfoam_model=SingleVertexSpinFoam(),
    spin_j=spin_j,
    main_layer_hidden_nodes=(50, 20),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.0005
)

In [ ]:
import os
from datetime import datetime


training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)


batch_size = int(1e3)
n_iterations = int(1e3)
# n_iterations = int(300)
fraction_of_training_from_back_trajectories = 0.0
evaluation_batch_size = int(10)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

In [ ]:
Max

Nth iteration: 1 Average Loss: 20.836390882130726
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 2.0459407847597513
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 1.4585196694603741
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 0.97864002385061055
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 0.52738381254082678
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 0.841442386344152

In [ ]:
0.99 Max

Nth iteration: 1 Average Loss: 25.94910689433053
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 3.7002476080396289
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 1.6780714611113932
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 0.868219155780009
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 1.2639663788832127
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 0.96857257687373355

In [ ]:
orig

Nth iteration: 1 Average Loss: 46.807858204553668
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 11.916711202519357
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 9.34522011892435
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 8.6954130018196949
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 7.1471832212631705
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 6.4898161948653925

In [ ]:
from core.runner import Runner
from core.environment import SingleVertexSpinFoam

spin_j = 3.5
sf_model = "single_vertex_model"

runner = Runner(
    spinfoam_model=SingleVertexSpinFoam(),
    spin_j=spin_j,
    main_layer_hidden_nodes=(30, 20),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.0005
)

In [ ]:
import os
from datetime import datetime


training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)


batch_size = int(1e3)
# n_iterations = int(1e3)
n_iterations = int(100)
fraction_of_training_from_back_trajectories = 0.75
evaluation_batch_size = int(10)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

In [ ]:
-52.23322188306788

In [ ]:
from core.runner import Runner
from core.environment import SingleVertexSpinFoam
import os
from datetime import datetime

list_of_ave_losses = []

for f in [0.0, 0.25, 0.5, 0.75, 1.0]:
    spin_j = 6.0
    sf_model = "single_vertex_model"

    runner = Runner(
        spinfoam_model=SingleVertexSpinFoam(),
        spin_j=spin_j,
        main_layer_hidden_nodes=(50, 20),
        branch1_hidden_nodes=(),
        branch2_hidden_nodes=(),
        activation="swish",
        exploration_rate=0.5,
        learning_rate=0.0005
    )
    training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
    directory_for_generated_samples = (
        f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
    )


    batch_size = int(1e3)
    n_iterations = int(1e4)
    fraction_of_training_from_back_trajectories = f
    evaluation_batch_size = int(2e3)
    generate_samples_every_m_training_samples = int(2e6)

    ave_losses = runner.train_agent(
        batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
        evaluation_batch_size, generate_samples_every_m_training_samples,
        directory_for_generated_samples
    )
    list_of_ave_losses.append(ave_losses)
    


In [ ]:
import matplotlib.pyplot as plt

for i, f in enumerate([0.0, 0.25, 0.5, 0.75, 1.0]):
    plt.plot(list_of_ave_losses[i], label=f)
    
plt.legend()

In [ ]:
import matplotlib.pyplot as plt

for i, f in enumerate([0.0, 0.25, 0.5, 0.75, 1.0]):
    plt.plot(list_of_ave_losses[i], label=f)
    
plt.legend()

In [ ]:
len(list_of_ave_losses)

# Star Model

In [ ]:
from core.runner import Runner
from core.environment import StarModelSpinFoam

spin_j = 3.5
sf_model = "star_model"

runner = Runner(
    spinfoam_model=StarModelSpinFoam(),
    spin_j=spin_j,
    main_layer_hidden_nodes=(128, 64, 64),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.0005
)


In [ ]:
import os
from datetime import datetime

training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)

batch_size = int(1e3)
n_iterations = int(1e4)
evaluation_batch_size = int(2e5)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

In [ ]:
# import numpy as np

# np.savetxt("Only forward trajectories.csv", ave_losses.numpy(), delimiter=",")


In [ ]:
import matplotlib.pyplot as plt

plt.plot(ave_losses.numpy())

In [ ]:
from core.runner import Runner
from core.environment import StarModelSpinFoam
import os
from datetime import datetime

list_of_ave_losses = []

for f in [0.0, 0.25, 0.5, 0.75, 1.0]:
    spin_j = 3.5
    sf_model = "star_model"

    runner = Runner(
        spinfoam_model=StarModelSpinFoam(),
        spin_j=spin_j,
        main_layer_hidden_nodes=(128, 64, 64),
        branch1_hidden_nodes=(),
        branch2_hidden_nodes=(),
        activation="swish",
        exploration_rate=0.5,
        learning_rate=0.0005
    )
    training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
    directory_for_generated_samples = (
        f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
    )


    batch_size = int(1e3)
    n_iterations = int(1e4)
    fraction_of_training_from_back_trajectories = f
    evaluation_batch_size = int(1e3)
    generate_samples_every_m_training_samples = int(1e7)

    ave_losses = runner.train_agent(
        batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
        evaluation_batch_size, generate_samples_every_m_training_samples,
        directory_for_generated_samples
    )
    list_of_ave_losses.append(ave_losses)
    


In [ ]:
import matplotlib.pyplot as plt

for i, f in enumerate([0.0, 0.25, 0.5, 0.75, 1.0]):
    plt.plot(list_of_ave_losses[i], label=f)
    
plt.legend()

In [ ]:
from core.runner import Runner
from core.environment import StarModelSpinFoam

spin_j = 3.5
sf_model = "star_model"

runner = Runner(
    spinfoam_model=StarModelSpinFoam(),
    spin_j=spin_j,
    main_layer_hidden_nodes=(128, 64, 64),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.0005
)


In [ ]:
import os
from datetime import datetime

training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")
directory_for_generated_samples = (
    f"generated_samples_during_training/{sf_model}/j={spin_j}/Training run on {training_run_datetime}"
)


batch_size = int(1e3)
# n_iterations = int(1e3)
n_iterations = int(100)
fraction_of_training_from_back_trajectories = 0.75
evaluation_batch_size = int(10)
generate_samples_every_m_training_samples = int(2e5)

ave_losses = runner.train_agent(
    batch_size, n_iterations, fraction_of_training_from_back_trajectories, 
    evaluation_batch_size, generate_samples_every_m_training_samples,
    directory_for_generated_samples
)

In [ ]:
0.99 Max * 6

Nth iteration: 1 Average Loss: 256.75957053640968
Nth iteration: 200 Trained on K samples: 200000 Average Loss: 27.373910871569272
Nth iteration: 400 Trained on K samples: 400000 Average Loss: 20.145873200509026
Nth iteration: 600 Trained on K samples: 600000 Average Loss: 13.870413800617818
Nth iteration: 800 Trained on K samples: 800000 Average Loss: 9.2659642975085212
Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 8.958290865724539

In [ ]:
import tensorflow as tf

6*2*tf.math.log(tf.reduce_max(tf.math.abs(runner.env.single_vertex_amplitudes)))

In [ ]:
-313.3993312984073*1.1

In [ ]:
6*2*tf.math.log(tf.reduce_sum(tf.math.abs(runner.env.single_vertex_amplitudes)))

In [ ]:
tf.reduce_max(tf.math.abs(runner.env.single_vertex_amplitudes))

In [ ]:
tf.reduce_sum(tf.math.abs(runner.env.single_vertex_amplitudes))

------------------------------------

# Previous tests

In [ ]:
from core.runner import Runner
import tensorflow as tf

grid_length = 8

runner = Runner(
    grid_length=grid_length,
    main_layer_hidden_nodes=(40, 30, 20),
    branch1_hidden_nodes=(),
    branch2_hidden_nodes=(),
    activation="swish",
    exploration_rate=0.5,
    learning_rate=0.001,
    environment_mode="spinfoam_vertex"
)

In [ ]:
half_batch_size = tf.constant(500)
n_iterations = tf.constant(int(1e4))
# n_iterations = tf.constant(500)

# evaluate_every_n_iterations = tf.constant(int(1e2))
evaluate_every_n_iterations = tf.constant(int(1e3))
evaluation_batch_size = tf.constant(int(5e4))
# evaluation_batch_sizes = tf.constant([int(1e4), int(5e4), int(1e5), int(5e5)])

ave_losses, distr_js_dists, agent_obss = runner.train_agent(
    half_batch_size, n_iterations, evaluate_every_n_iterations, evaluation_batch_size
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_results(
    train_iters, eval_iters, losses, js_dists, obs, theo_obs, figsize=(10, 8)
):
    fig, ax = plt.subplots(3, 1, figsize=figsize)

    ax[0].scatter(train_iters, losses, marker='.', alpha=0.1, c='b', s=5)
    ax[0].scatter(eval_iters, losses[eval_iters-1], marker='o', c='r', s=50)
    ax[0].set_ylabel('Training Loss')
    ax[0].set_xlabel('Nth Iteration')


    ax[1].plot(eval_iters, js_dists[1:], marker='o')
    ax[1].plot(0, js_dists[0], marker='*', c='r')
    ax[1].set_ylabel('Jensen–Shannon Distance')
    ax[1].set_xlabel('Nth Iteration')
    ax[1].set_ylim(0, 1)

    ax[2].plot(eval_iters, obs[1:], marker='o')
    ax[2].plot(0, obs[0], marker='*', c='r')
    ax[2].fill_between(
        x=[0, eval_iters[-1]], y1=1.1*theo_obs, y2=0.9*theo_obs, 
        alpha=0.1, color='k'
    )
    ax[2].hlines(theo_obs, 0, eval_iters[-1], linestyles='dashed', colors='k')


    ax[2].set_ylabel('Average Dihedral Angle')
    ax[2].set_xlabel('Nth Iteration')
    ax[2].set_ylim(-0.95, 0.6)
    
    plt.tight_layout()
    

training_iterations = np.arange(n_iterations) + 1
evaluation_iterations = np.arange(
    evaluate_every_n_iterations, 
    n_iterations+1,
    evaluate_every_n_iterations
)

training_losses = ave_losses.numpy()

js_distances = distr_js_dists.numpy()

observables = agent_obss.numpy()
theoretical_obs = runner.env.theoretical_ave_dihedral_angle.numpy()


plot_results(
    training_iterations, evaluation_iterations, training_losses, 
    js_distances, observables, theoretical_obs, figsize=(10, 8)
)

In [ ]:
import matplotlib.pyplot as plt

def plot_results(
    train_iters, eval_iters, losses, distr_errs, obs, theo_obs, eval_sizes, 
    figsize=(10, 8), colors=['b', 'r', 'g', 'y']
):
    fig, ax = plt.subplots(3, 1, figsize=figsize)

    ax[0].scatter(train_iters, losses, marker='.', alpha=0.1, c='b', s=5)
    ax[0].scatter(eval_iters, losses[eval_iters-1], marker='o', c='r', s=50)
    ax[0].set_ylabel('Training Loss')
    ax[0].set_xlabel('Nth Iteration')

    for ind in range(len(eval_sizes)):
        ax[1].plot(
            eval_iters, distr_errs[1:, ind], 
            label=f"Evaluation batch size = {eval_sizes[ind]}",
            marker='o', color=colors[ind]
        )
        ax[1].plot(0, distr_errs[0, ind], marker='*', color=colors[ind])

#         frac_error = np.abs(obs - theo_obs) / np.abs(theo_obs)
        
        ax[2].plot(
            eval_iters, obs[1:, ind], 
            label=f"Evaluation batch size = {eval_sizes[ind]}",
            marker='o', color=colors[ind]
        )
        ax[2].plot(0, obs[0, ind], marker='*', color=colors[ind])
        ax[2].hlines(
            theo_obs, 0, eval_iters[-1], 
            linestyles='dashed', colors='k'
        )

    ax[1].set_ylabel('Average Distribution Error')
    ax[1].set_xlabel('Nth Iteration')
    ax[1].legend()
    
    ax[2].set_ylabel('Average Dihedral Angle')
    ax[2].set_xlabel('Nth Iteration')
    ax[2].set_ylim(-0.95, 0.55)
    ax[2].legend(prop={"size": 7})

    plt.tight_layout()
    
plot_results(
    training_iterations, evaluation_iterations, training_losses, 
    distribution_errors, observables, theoretical_obs, eval_batch_sizes
)

In [ ]:
frac_error = np.abs(observables[1:, -1] - theoretical_obs) / np.abs(theoretical_obs)
plt.plot(frac_error)

In [ ]:

@tf.function(input_signature=[
    tf.TensorSpec(shape=None, dtype=tf.int32),
    tf.TensorSpec(shape=None, dtype=tf.int32)
])
def test_fn(n_iter, sizes):
    observables = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    for i in tf.range(n_iter):
        
        obs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        j = 0
        for size in sizes:
            obs = obs.write(j, tf.cast(i + size, dtype=tf.float32))
            j += 1

        observables = observables.write(i, obs.stack())
        
    observables = observables.stack()
    return observables
        

In [ ]:
test_fn(tf.constant(10), tf.constant([500, 1000]))

In [ ]:
import numpy as np

train_iters = np.arange(n_iterations) + 1
eval_iters = np.arange(
    evaluate_every_n_iterations, 
    n_iterations+1,
    evaluate_every_n_iterations
)

losses = ave_losses.numpy()
plt.scatter(train_iters, losses, marker='.', alpha=0.1, c='b', s=5)
plt.scatter(eval_iters, losses[eval_iters-1], marker='o', c='r', s=50)

In [ ]:
obs = agent_obss.numpy()
theo_obs = runner.env.theoretical_ave_dihedral_angle.numpy()

plt.plot(eval_iters, obs)
plt.hlines(
    theo_obs, eval_iters[0], eval_iters[-1], linestyles='dashed', colors='k'
)

In [ ]:
frac_error = np.abs(obs - theo_obs) / np.abs(theo_obs)

plt.plot(eval_iters, frac_error)
plt.hlines(
    0, eval_iters[0], eval_iters[-1], linestyles='dashed', colors='k'
)

In [ ]:
train_iters = tf.range(n_iterations) + 1
eval_iters = tf.range(
    start=evaluate_every_n_iterations, 
    limit=n_iterations+1, delta=evaluate_every_n_iterations
)

plt.scatter(train_iters.numpy(), ave_losses.numpy(), marker='.', alpha=0.4)
plt.scatter(eval_iters.numpy(), ave_losses.numpy(), marker='.', alpha=0.4)

In [ ]:
ave_losses

In [ ]:



tf.gather(ave_losses, eval_iters - 1)

In [ ]:
tf.where(tf.math.equal(tf.math.floormod(train_iters, evaluate_every_n_iterations), 0))

In [ ]:
import matplotlib.pyplot as plt

sq_a = runner.env.squared_amplitudes.numpy().ravel()
rewards = runner.env.rewards.numpy().ravel()

plt.hist(sq_a, bins=200);
plt.ylim(0, 10)

In [ ]:
plt.hist(rewards, bins=200);
plt.ylim(0, 10)

In [ ]:
import time

half_batch_size = tf.constant(500)
n_iterations = tf.constant(int(2e3))
# n_iterations = tf.constant(500)
evaluate_every_n_iterations = tf.constant(int(1e2))
evaluation_batch_size = tf.constant(int(5e4))

start = time.time()
ave_losses, ave_distr_errors, agent_obs = runner.train_agent(
    half_batch_size, n_iterations, evaluate_every_n_iterations, evaluation_batch_size
)
time_elapsed = time.time() - start
print(f"Time Elapsed: {time_elapsed} s")
print(f"Average loss of last iteration: {float(ave_losses[-1])}")

In [ ]:
# import matplotlib.pyplot as plt

plt.plot(ave_losses.numpy())

In [ ]:
plt.plot(ave_distr_errors.numpy())
# ave_distr_errors

In [ ]:
plt.plot(agent_obs.numpy())
plt.hlines(
    runner.env.theoretical_ave_dihedral_angle.numpy(),
    0, len(agent_obs) - 1, linestyles='dashed', colors='k'
)

In [ ]:
runner.env.theoretical_ave_dihedral_angle.numpy()

In [ ]:
env_distr = runner.env.rewards / tf.math.reduce_sum(runner.env.rewards)

In [ ]:
import numpy as np

def _plot_l1_errors_per_probability_interval(agent_prob, env_prob, 
#                                              filename, 
                                             n_intervals
                                            ):
    result = agent_prob.ravel()
    expected = env_prob.ravel()
    max_expected = expected.max()
    expected_pcts = expected * 100 / max_expected

    interval_edges = np.linspace(0, 100, n_intervals + 1)
    starts = interval_edges[:-1]
    ends = interval_edges[1:]

#     frac_l1_errors = np.abs(result - expected) / max_expected
    l1_errors = np.abs(result - expected)
    errors = []
    labels = []
    for s, e in zip(starts, ends):
        inds = np.where((s < expected_pcts) & (expected_pcts <= e))
#         errors_per_interval = frac_l1_errors[inds]
        errors_per_interval = l1_errors[inds]
        if len(errors_per_interval):
            interval_label = f"{s:.2f}% < pct <= {e:.2f}%"
            max_expected_per_interval = expected[inds].max()
            errors.append(errors_per_interval / max_expected_per_interval)
            labels.append(interval_label)

#     print(errors)
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.violinplot(errors)
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels, rotation=15)
    ax.set_ylim(0, 1.0)
    ax.set_ylabel("Fractional L1 Errors")
    ax.set_xlabel("Percentage of max theoretical probability")
    plt.tight_layout()
#     fig.savefig(f"./plot_results/{filename}")
    return errors

In [ ]:
batch_size = tf.constant(int(5e4))

agent_distr = runner.get_normalized_agent_sample_distribution(batch_size)

grid_dim = 5
n_unique_samples = tf.shape(tf.where(agent_distr))[0]
frac_unique_coords = n_unique_samples / (grid_length ** grid_dim)
print(f"Percentage of coordinates sampled uniquely: {float(frac_unique_coords*100):.2f}%")

In [ ]:
errors = _plot_l1_errors_per_probability_interval(
    agent_distr.numpy(), env_distr.numpy(), n_intervals=100
)

In [ ]:
batch_size = tf.constant(int(1e5))

agent_distr = runner.get_normalized_agent_sample_distribution(batch_size)

grid_dim = 5
n_unique_samples = tf.shape(tf.where(agent_distr))[0]
frac_unique_coords = n_unique_samples / (grid_length ** grid_dim)
print(f"Percentage of coordinates sampled uniquely: {float(frac_unique_coords*100):.2f}%")

In [ ]:
errors = _plot_l1_errors_per_probability_interval(
    agent_distr.numpy(), env_distr.numpy(), n_intervals=100
)

In [ ]:
batch_size = tf.constant(int(5e5))

agent_distr = runner.get_normalized_agent_sample_distribution(batch_size)

grid_dim = 5
n_unique_samples = tf.shape(tf.where(agent_distr))[0]
frac_unique_coords = n_unique_samples / (grid_length ** grid_dim)
print(f"Percentage of coordinates sampled uniquely: {float(frac_unique_coords*100):.2f}%")

In [ ]:
errors = _plot_l1_errors_per_probability_interval(
    agent_distr.numpy(), env_distr.numpy(), n_intervals=100
)